In [22]:
from pydataset import data
import pandas as pd
from acquire import getData
# import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt

In [2]:
df_iris = data('iris')
df_iris.head(3)


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa


In [3]:
df_iris.shape

(150, 5)

In [4]:
df_iris.columns

Index(['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width',
       'Species'],
      dtype='object')

In [5]:
df_iris.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150 entries, 1 to 150
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Sepal.Length  150 non-null    float64
 1   Sepal.Width   150 non-null    float64
 2   Petal.Length  150 non-null    float64
 3   Petal.Width   150 non-null    float64
 4   Species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 7.0+ KB


In [6]:
df_iris.describe()


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [7]:
df_excel = pd.read_csv('Telco_plus.csv')
df_excel.head(2)

,customer_id,gender,is_senior_citizen,partner,dependents,phone_service,internet_service,contract_type,payment_type,monthly_charges,total_charges,churn,tenure
0,0002-ORFBO,Female,0,Yes,Yes,1,1,1,Mailed check,65.6,593.3,No,9.0
1,0003-MKNFE,Male,0,No,No,2,1,0,Mailed check,59.9,542.4,No,9.1


In [8]:
df_excel_sample = df_excel.head(100) 
# df2 = df_excel.iloc[:100,:] #ALT


In [9]:
#number of rows in original dataframe
df_excel.shape[0]

7049

In [10]:
#first 5 columns
df_excel.columns[0:5]

Index(['customer_id', 'gender', 'is_senior_citizen', 'partner', 'dependents'], dtype='object')

In [11]:
#print columns that are objects
for i in df_excel.columns:
    if df_excel[i].dtype == object: print(i)

customer_id
gender
partner
dependents
payment_type
churn


In [12]:
#ALT print column objects
df_excel.select_dtypes(object).columns

Index(['customer_id', 'gender', 'partner', 'dependents', 'payment_type',
       'churn'],
      dtype='object')

In [13]:
#Calculate range for numeric variables (doing all int/float, not filtering out those that are actual categories)
for i in df_excel.columns:
    if df_excel[i].dtype == int or df_excel[i].dtype == float:
        rng = df_excel[i].max() - df_excel[i].min()
        print(f'{i} range: {rng}')


is_senior_citizen range: 1
phone_service range: 2
internet_service range: 2
contract_type range: 2
monthly_charges range: 100.5
total_charges range: 8666.0
tenure range: 79.3


In [14]:
#ALT way to get range:
df_excel.select_dtypes(include=[int,float]).max()-df_excel.select_dtypes(include=[int,float]).min()

is_senior_citizen       1.0
phone_service           2.0
internet_service        2.0
contract_type           2.0
monthly_charges       100.5
total_charges        8666.0
tenure                 79.3
dtype: float64

In [15]:
#From Google Sheets
url="https://docs.google.com/spreadsheets/d/1Uhtml8KY19LILuZsrDtlsHHDC9wuDGUSe8LTEwvdI5g/edit#gid=341089357"
url = url.replace('edit#gid=','export?format=csv&gid=')
df_google = pd.read_csv(url)
df_google.head(3)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [16]:
df_google.shape

(891, 12)

In [17]:
df_google.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [18]:
df_google.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [19]:
cats = ['Survived','Pclass','Sex','SibSp','Parch','Embarked']
for i in cats:
    print(f'{i} categories: {df_google[i].unique()}')

Survived categories: [0 1]
Pclass categories: [3 1 2]
Sex categories: ['male' 'female']
SibSp categories: [1 0 3 4 2 5 8]
Parch categories: [0 1 2 5 3 4 6]
Embarked categories: ['S' 'C' 'Q' nan]


In [23]:
t_df = getData('titanic')


In [25]:
help(getData)

Help on function getData in module acquire:

getData(db, ds=            filename      db_name  \
titanic  titanic.csv   titanic_db   
iris        iris.csv      iris_db   
telco      telco.csv  telco_churn   

                                                                                               sql  
titanic                                                                  SELECT * FROM passengers;  
iris                                    SELECT * FROM measurements JOIN species USING(species_id);  
telco    \nSELECT * FROM customers\nJOIN internet_service_types USING(internet_service_type_id)...  )
    Returns: Pandas dataframe of desired dataset
    Required Parameter: dataset name
    Supported datasets: 
      titanic
      iris
      telco
    
    Checks working directory for csv file of data.
    If no CSV, retrieves dataset from Codeup DB and stores a local csv file

